【実行に関して注意点】

- ランタイムはGPUにする
- 言語モデルのサイズが2GBあるので、それ以上のディスク容量が空いていることを要確認

# 【概要】

SttSの由来（Summarisation, Translation, and Text to Speechの略称。）筆者が命名した。

- Summarisation：要約
- Translation：翻訳
- Text to Speech：音声合成

これらをすべて実行するコードを作成した。

## 【作成方針】

- とにかく無料で使えるもので構成している。
- 「要約」や「翻訳」については、有料でもっと精度の良いものなどがあるので、精度を上げたい場合は該当箇所を書き換えれば実行できる。

## 【今後】
- GPT-3系（GPT3.5）を使うなどすれば、諸々の精度が向上すると思われる。無料枠があると言えど、気軽にバンバン使えるわけではなく、開発に慎重さを求められてしまうのが好きではないので、今回は一旦無料のAPI群ですべてを構成した。
- 画像ファイルやPDFからのOCRを組み込めば、プロジェクト資料などの要約を手に入れることができる
- webスクレイピングと組み合わせれば、本文の翻訳や要約した情報を手に入れることができる。

## 【インプットデータ】
日本語か英語のテキストファイル


## 【アウトプットデータ】
日本語または英語で発話されている、wavファイル

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ファイル名の設定
SRC_FILE_NAME    = "src_text.txt" # 入力テキストファイルの名前
OUTPUT_WAV_NAME  = "output.wav"   # 出力wavファイルの名前
OUTPUT_TEXT_NAME = "output.txt"   # 出力テキストファイルの名前


# 動作周りの設定
IS_SUMMARISE = True # 入力した文章を要約したいときは、"True"。要約しないのであれば、"False"
IS_JAPANISE  = False # 音声合成を英語にするのか、日本語にするのか設定。日本語にしたいときは、"True"、英語の場合は、"False"

In [ ]:
# 各種ファイルのディレクトリ設定

DIR_INPUT  = "/content/drive/MyDrive/colab/SttS/input/"
DIR_OUTPUT = "/content/drive/MyDrive/colab/SttS/output/"

In [ ]:
# 各種ライブラリのインストール

# 言語判定のライブラリ
!pip install langid


# ここのページでPyTorchのインストールコマンドをセットする
# ページ：https://pytorch.org/get-started/locally/
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116


# 次の2つをインストールする
# ・Transformers
# ・SentencePiece
!pip install transformers
!pip install sentencepiece


# 翻訳のライブラリ
!pip install googletrans==4.0.0-rc1


# Googleの text to speechのインストール
!pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941188 sha256=121d5723f1b8bbe4c4dac3af669a69a4c61a8decbf7ac49b10994e70849c8e19
  Stored in directory: /root/.cache/pip/wheels/c5/01/a4/0160c55074707b535a6757a541842817d530d8080ca943a107
Successfully built langid
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.4 MB/s eta 0:00:00
Looking in indexes:

In [ ]:
# 各種ライブラリのインポート

# 言語判定のライブラリ
import langid


# 要約関連のライブラリ
from transformers import PegasusForConditionalGeneration, PegasusTokenizer 
import torch 


# 翻訳関連のライブラリ
from googletrans import Translator


# 音声発話のためのライブラリ
from gtts import gTTS


# Jupyterファイル上でwavファイルを再生できるようにするためのライブラリ
from IPython.display import Audio


import pandas as pd
import numpy as np

In [ ]:
with open(DIR_INPUT + SRC_FILE_NAME) as f:
    src_text = f.read()

In [ ]:
# 言語判定の関数
# 日本語であれば"ja"、英語であれば、"en"
def ClassifyLanguage(text = "default text"):
  """
  input:text (str)
  output:what language is source text data (str)
  """
  result = langid.classify(text)

  return result[0]

In [ ]:
# 日本語→英語への翻訳
def MakeTextEnglish(text = "デフォルトの文章", src_lang = "ja"):
  """
  input: Japanese text (str)
  output: English text (str)
  """
  tr = Translator()

  while True:
    try:
      result = tr.translate(text, dest="en").text
      break
    except Exception as e:
      tr = Translator()

  return result

In [ ]:
# 要約を実行する
def SummarisationText(text = "default text"):
  """
  input: English text (str)
  output: summarised English text (str)
  """
  # モデル読み込み 
  model_name = 'google/pegasus-large'
  tokenizer  = PegasusTokenizer.from_pretrained(model_name)

  # デバイスの設定
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # 要約の実行
  model     = PegasusForConditionalGeneration.from_pretrained(model_name).to(device) 
  batch     = tokenizer(text, 
                        truncation=True,
                        padding='longest',
                        return_tensors="pt",
                        max_length = 1024).to(device) 
  translated = model.generate(**batch)
  list_summarised_text   = tokenizer.batch_decode(translated, skip_special_tokens=True)

  result = list_summarised_text[0] # 今回は1つだけの文章だからインデックスはゼロ

  return result

In [ ]:
# 英語→日本語への翻訳
def MakeTextJapanese(text = "default text", src_lang = "en"):
  """
  input: English text (str)
  output: Japanese text (str)
  """
  text = text.replace("\n", " ")

  tr = Translator()

  while True:
    try:
      result = tr.translate(text, dest="ja").text
      break
    except Exception as e:
      tr = Translator()

  return result

In [ ]:
# テキストデータから音声ファイル(wavファイル)を作成する
def GenerateWavFile(text_to_say = "こんにちは", language = "ja"):
  gtts_object = gTTS(text = text_to_say, 
                     lang = language,
                     slow = False)

  gtts_object.save(DIR_OUTPUT + OUTPUT_WAV_NAME)

In [ ]:
# 要約でたまにバグる（同じ単語が永遠と続いてしまう）ので、
# それが起きても、問題がないように、
# 連続して同じ単語が続いた場合は一単語に直す
def DeleteDuplicatedText(text = "a a a a"):
  #text = text.replace("\n", " ")
  data = text.split(' ')

  df = pd.Series(data)

  # 一つ手前の要素と異なる要素だけを抜き出す
  df = df[df != df.shift(1)]

  list_ = df.values.tolist()

  result = ' '.join(list_)

  return result

In [ ]:
str_language = ClassifyLanguage(text = src_text)

In [ ]:
is_English = str_language == 'en'

# そもそもの言語が英語であれば、翻訳する必要ないので、
# 要約対象のデータとして"target_smmrs_text"に格納する
if is_English:
  target_smmrs_text = src_text
else: #英語以外の言語であれば、英語に翻訳する
  translated_text   = MakeTextEnglish(text = src_text, src_lang = str_language)
  target_smmrs_text = translated_text

In [ ]:
# 要約の実行

if IS_SUMMARISE:
  target_trsnslt_text = SummarisationText(text = target_smmrs_text)
else:
  target_trsnslt_text = target_smmrs_text

target_trsnslt_text = DeleteDuplicatedText(target_trsnslt_text)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# 音声合成の言語を設定して、その言語のテキストファイルを生成する

if IS_JAPANISE:
  target_spch_text = MakeTextJapanese(text = target_trsnslt_text)
  GenerateWavFile(text_to_say = target_spch_text, language = "ja")
else:
  target_spch_text = target_trsnslt_text
  GenerateWavFile(text_to_say = target_spch_text, language = "en")

In [ ]:
Audio(DIR_OUTPUT + OUTPUT_WAV_NAME)

In [ ]:
with open(DIR_OUTPUT + OUTPUT_TEXT_NAME, mode='w') as f:
  f.write(target_spch_text)